TODO
- change units to real units
- work on visualizations
- build shared data frame of stance vs swing phases per mouse
- build shared data frame of mobile vs immobile phases per mouse
- randomly sample 

In [1]:
pip install -q ..

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mouse_gait_analysis

from mouse_gait_analysis.io import *
from mouse_gait_analysis.utils import *

from mouse_gait_analysis.utils import VideoAnalysis

In [3]:
video_analysis = VideoAnalysis(
    "/Users/andretelfer/shared/thea/trial-analysis/AccA19.mp4",
    "/Users/andretelfer/shared/thea/trial-analysis/AccA19DLC_resnet50_A19allMar17shuffle1_1030000.h5")

In [4]:
video_analysis.keypoints = filter_likelihood(video_analysis.keypoints, 0.6)
video_analysis.keypoints = filter_distance_traveled(video_analysis.keypoints, 20, 5)



bodyparts        nose                              chest              \
coords              x           y likelihood           x           y   
0                 NaN         NaN        NaN         NaN         NaN   
1                 NaN         NaN        NaN         NaN         NaN   
2                 NaN         NaN        NaN         NaN         NaN   
3                 NaN         NaN        NaN         NaN         NaN   
4                 NaN         NaN        NaN         NaN         NaN   
...               ...         ...        ...         ...         ...   
65941      976.271729  279.264252   0.999474  958.020874  283.877411   
65942      975.938049  279.080078   0.999180  957.738403  283.906464   
65943      975.209351  279.159668   0.998662  957.169678  283.879120   
65944      975.024292  279.338348   0.998043  957.124084  283.807922   
65945      975.150513  279.051880   0.997842  957.271667  283.686493   

bodyparts               tailbase                        left_front_paw  ...  \
coords    likelihood           x           y likelihood              x  ...   
0                NaN         NaN         NaN        NaN            NaN  ...   
1                NaN         NaN         NaN        NaN            NaN  ...   
2                NaN         NaN         NaN        NaN            NaN  ...   
3                NaN         NaN         NaN        NaN            NaN  ...   
4                NaN         NaN         NaN        NaN            NaN  ...   
...              ...         ...         ...        ...            ...  ...   
65941       0.999419  885.815247  282.101593        1.0     969.093201  ...   
65942       0.999339  885.804260  282.081085        1.0     968.578979  ...   
65943       0.999398  885.639709  282.078064        1.0     969.773376  ...   
65944       0.999388  885.683960  282.079620        1.0     969.664429  ...   
65945       0.999351  885.567383  282.101990        1.0     969.607849  ...   

bodyparts  left_toes left_back_paw                         right_toes  \
coords    likelihood             x           y likelihood           x   
0                NaN           NaN         NaN        NaN         NaN   
1                NaN           NaN         NaN        NaN         NaN   
2                NaN           NaN         NaN        NaN         NaN   
3                NaN           NaN         NaN        NaN         NaN   
4                NaN           NaN         NaN        NaN         NaN   
...              ...           ...         ...        ...         ...   
65941       0.999732    934.540039  295.510284   0.999999  947.110657   
65942       0.999656    934.427917  295.453217   0.999999  947.008423   
65943       0.999564    934.370605  295.346771   0.999999  946.934021   
65944       0.999545    934.363770  295.390198   0.999999  946.982422   
65945       0.999576    934.306152  295.420258   0.999999  947.039062   

bodyparts                        right_back_paw                         
coords              y likelihood              x           y likelihood  
0                 NaN        NaN            NaN         NaN        NaN  
1                 NaN        NaN            NaN         NaN        NaN  
2                 NaN        NaN            NaN         NaN        NaN  
3                 NaN        NaN            NaN         NaN        NaN  
4                 NaN        NaN            NaN         NaN        NaN  
...               ...        ...            ...         ...        ...  
65941      238.978867   0.999989     922.942627  251.410690   0.999999  
65942      239.110199   0.999989     922.789246  251.458282   0.999999  
65943      238.980164   0.999990     922.717896  251.483856   0.999999  
65944      239.025452   0.999989     922.759338  251.442795   0.999999  
65945      238.971771   0.999989     922.795715  251.402100   0.999999  

[65946 rows x 27 columns]